In [44]:
import numpy as np
import pandas as pd
from dotenv import load_dotenv
import os
import alpaca_trade_api as tradeapi
from trade import trade_action
from datetime import datetime, timedelta

In [45]:
def market_data( start, end, tickers ):
    load_dotenv()
    alpaca_api_key = os.getenv('ALPACA_API_KEY')
    alpaca_secret_key = os.getenv('ALPACA_SECRET_KEY')

    # Create the Alpaca API object
    alpaca_api = tradeapi.REST(
       alpaca_api_key,
       alpaca_secret_key,
       api_version = 'v2'
    )
    
    # Set timeframe to "1Day" for Alpaca API
    timeframe = "1Day"
    
    start_date = pd.Timestamp(start,tz='America/New_York')
    end_date = pd.Timestamp(end,tz='America/New_York')
           
    # Get number_of_years' worth of historical data for tickers
    data_df = alpaca_api.get_bars(
        tickers,
        timeframe,
        start = start_date.isoformat(),
        end = end_date.isoformat()
    ).df
    if len(data_df) == 0:
        return []
    return data_df

#df_market = market_data( '2023-08-01', '2023-08-21', tickers=['TSLA'] )

## Plan
* Get all ticker data for a year
* Get one ticker, get signal column
* Slice backwards, 15 for pred, 30 for train
* Scale train
* Rolling train, 30 at a time, predict one day at a time, till the end
* Store the prediction in my instr_df

In [46]:
today = datetime.today().date()
tickers = ['AAPL', 'AMZN', 'MSFT', 'GOOG']
date_from = today - timedelta(days=365)
data_df = market_data(date_from, today, tickers)
aapl_df = data_df[data_df['symbol']=='AAPL']
aapl_cycle = aapl_df.iloc[-45:,:]
#aapl_cycle
# get signals
aapl_signal = aapl_cycle.loc[:,['close']]
aapl_signal['daily_return'] = aapl_signal['close'].pct_change()
aapl_signal['lagged_daily_return'] = aapl_signal['daily_return'].shift(1)
aapl_signal['twice_lagged'] = aapl_signal['daily_return'].shift(2)

In [47]:
aapl_signal[['daily_return','lagged_daily_return','twice_lagged']].hvplot(frame_width=700)

:NdOverlay   [Variable]
   :Curve   [timestamp]   (value)

In [36]:
aapl_signal['signal'] = np.where((aapl_signal['lagged_daily_return']>0) &
                                 (aapl_signal['twice_lagged']>0), -1, 
                                 np.where((aapl_signal['lagged_daily_return']<0) &
                                          (aapl_signal['twice_lagged']<0), 1, 0 
                                         )
                                )
aapl_signal

,close,daily_return,lagged_daily_return,twice_lagged,signal
timestamp,,,,,
2023-06-26 04:00:00+00:00,185.270,NaN,NaN,NaN,0
2023-06-27 04:00:00+00:00,188.060,0.015059,NaN,NaN,0
2023-06-28 04:00:00+00:00,189.250,0.006328,0.015059,NaN,0
2023-06-29 04:00:00+00:00,189.590,0.001797,0.006328,0.015059,-1
2023-06-30 04:00:00+00:00,193.880,0.022628,0.001797,0.006328,-1
2023-07-03 04:00:00+00:00,192.460,-0.007324,0.022628,0.001797,-1
2023-07-05 04:00:00+00:00,191.330,-0.005871,-0.007324,0.022628,0
2023-07-06 04:00:00+00:00,191.810,0.002509,-0.005871,-0.007324,1
2023-07-07 04:00:00+00:00,190.680,-0.005891,0.002509,-0.005871,0


In [42]:
X = aapl_cycle.drop('symbol',axis=1)
y = aapl_signal['signal']

In [43]:
from sklearn.preprocessing import StandardScaler
stsc = StandardScaler()
stsc.fit(X)
X_scaled = stsc.transform(X)

In [ ]:
def train(df, number_of_times=2, dates_per_time=45):
    end_date = df.index.max()
    pred_start = 
    

In [3]:
# build a random instruction dataframe example:
#df = pd.DataFrame([['2023-08-24', 'TSLA', 1], 
#                   ['2023-08-25', 'AAPL', 1])
tickers = ['TSLA','AAPL','MSFT','AMZN']
instrs = np.random.randn(50, 1)

def sample_df():
    instri = 0
    start_date = pd.to_datetime('2023-08-01').normalize()

    # using alpaca data Aug 01 - Aug 21 for the dates to avoid non-trading days
    df = pd.DataFrame()
    for index, data in df_market.iterrows():
        if instrs[instri] > 0:
            instr = 1
        else:
            instr = -1
        instri += 1
        one_row = pd.DataFrame([[index.date(), tickers[np.random.randint(4)], instr]])
        df = pd.concat([df,one_row], join='outer')
    return df
    

In [8]:
final_worths = []
for n in range(500):
    final_worths.append(trade_action( sample_df(), verbose=0 ))

In [9]:
pd.DataFrame(final_worths).hvplot(kind='hist')

:Histogram   [0]   (0_count)

In [10]:
trade_action(sample_df())

On 2023-08-01 trade TSLA
{'action': 'buy', 'price': 263.51, 'bal': 7628.41, 'share': 9, 'status': 0, 'msg': 'success'}
On 2023-08-02 trade TSLA
{'action': 'sell', 'price': 254.92, 'bal': 9922.69, 'share': 9, 'status': 0, 'msg': 'success'}
On 2023-08-03 trade AMZN
{'action': 'none', 'price': 128.16, 'bal': 9922.69, 'share': 0, 'status': -1, 'msg': 'no share to sell'}
On 2023-08-04 trade MSFT
{'action': 'buy', 'price': 330.51, 'bal': 7609.12, 'share': 7, 'status': 0, 'msg': 'success'}
On 2023-08-07 trade MSFT
{'action': 'sell', 'price': 329.28, 'bal': 9914.08, 'share': 7, 'status': 0, 'msg': 'success'}
On 2023-08-08 trade TSLA
{'action': 'buy', 'price': 248.27, 'bal': 7431.38, 'share': 10, 'status': 0, 'msg': 'success'}
On 2023-08-09 trade AMZN
{'action': 'buy', 'price': 138.81, 'bal': 4932.8, 'share': 18, 'status': 0, 'msg': 'success'}
On 2023-08-10 trade AMZN
{'action': 'sell', 'price': 138.88, 'bal': 7432.64, 'share': 18, 'status': 0, 'msg': 'success'}
On 2023-08-11 trade MSFT
{'actio

:Layout
   .NdOverlay.I  :NdOverlay   [Variable]
      :Curve   [date]   (value)
   .NdOverlay.II :NdOverlay   [Variable]
      :Curve   [date]   (value)